<h1>**Приклад візуального аналізу даниx**</h1>

<h3 style="text-align: justify">Налаштуємо оточення: імпортуємо всі необхідні бібліотеки і налаштуємо дефолтне відображення картинок. </h3>

In [ ]:
# Відображення графіків у Jupyter notebook
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
# формат графіків - svg 
%config InlineBackend.figure_format = 'svg' 

# Збільшуємо стандартний розмір графіків
from pylab import rcParams
rcParams['figure.figsize'] = 8,5
# Імпортуємо Pandas i Numpy
import pandas as pd
import numpy as np

<h3 style="text-align: justify">Зчитуємо в DataFrame знайомі нам з першої лабораторної роботи дані по відтоку клієнтів телефонного оператору.</h3>

In [ ]:
df = pd.read_csv("../input/telecom-churn/telecom_churn.csv")

<h3 style="text-align: justify">Перевіримо, чи все нормально зчиталося − подивимось на перші 5 рядків (метод *head*).</h3>

In [ ]:
df.head()

<h3 style="text-align: justify">Число рядків (клієнтів) і стовпців (ознак):</h3>

In [ ]:
df.shape

<h3 style="text-align: justify">Подивимось на ознаки і переконаємося, що пропусків в жодному з них немає − скрізь по 3333 записів.</h3>

In [ ]:
df.info()

<h3 style="text-align: justify">Подивимось на розподіл цільової ознаки − відтоку клієнтів.</h3>

In [ ]:
df['churn'].value_counts()

In [ ]:
df['churn'].value_counts().plot(kind='bar', label='churn')
plt.legend()
plt.title('Розподіл відтоку клієнтів');

<h3>Виділимо наступні групи ознак (серед усіх крім *churn*):</h3>
<span style="font-size: 16px">
* бінарні: *international plan, voice mail plan;*
* категоріальні: *state, phone number;*
* порядкові: *customer service calls;*
* кількісні: всі інші.</span>

<h3 style="text-align: justify">Подивимось на кореляції кількісних ознак. За розфарбованою матрицею кореляцій видно, що такі ознаки як *Total day charge* рахуються по обговорених хвилинах (*Total day minutes*). Тобто 4 ознаки можна викинути, вони не несуть корисної інформації, а саме *Total day charge*, *Total eve charge*, *Total night charge*,*Total intl charge*.</h3>

In [ ]:
corr_matrix = df.drop(['state', 'international plan', 'voice mail plan',
                      'area code'], axis=1).corr()

In [ ]:
sns.heatmap(corr_matrix);

<h3 style="text-align: justify">Тепер подивимося на розподіл усіх цікавих для нас кількісних ознак. На бінарні / категоріальні / порядкові ознаки будемо дивитися окремо.</h3>

In [ ]:
features = list(set(df.columns) - set(['state', 'international plan', 'voice mail plan',  'area code',
                                       'churn', 'phone_number', 'total day charge', 'total eve charge', 
                                       'total night charge','total intl charge','phone number']))

In [ ]:
df[features].hist(figsize=(20,12));

<h3 style="text-align: justify">Бачимо, що більшість ознак розподілені нормально. Винятки − число дзвінків в сервісний центр (*Customer service calls*) (тут більше підходить пуасонівський розподіл) і число голосових повідомлень (*Number vmail messages*, пік в нулі, тобто це ті, у кого голосова пошта не підключена). Також зміщений розподіл числа міжнародних дзвінків (*Total intl calls*).</h3>

<h3 style="text-align: justify">Ще корисно побудувати ось такі картинки, де на головній діагоналі малюється розподіл ознак, а поза головною діагоналлю − діаграми розсіювання для пар ознак. Буває, що це призводить до  деяких висновків, але в даному випадку все приблизно зрозуміло, без сюрпризів.</h3>

<h3 style="color: red;text-align: justify">Для виконання дана операція вимагає багато обчислювальних ресурсів. Тому дочекайтесь коли в папці output/kaggle/working з'явиться зображення pairplot.png (інструкція на [картинці](https://drive.google.com/open?id=14YgIupHs-HRpYDsqkpOIsuv3aklFwuau)).</h3>

In [ ]:
# Для запуску розкоментуйте цей код
#sns_pairplot = sns.pairplot(df[features + ['churn']], hue='churn');
#ns_pairplot.savefig('pairplot.png')

<h3 style="text-align: justify">**Далі подивимося, як ознаки пов'язані з цільовою − з відтоком**.</h3>

<h3 style="text-align: justify">Побудуємо boxplot-и, що описують статистику розподілу кількісних ознак в двох групах: серед постійних клієнтів і клієнтів, які пішли.</h3>

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16, 10))

for idx, feat in  enumerate(features):
    sns.boxplot(x='churn', y=feat, data=df, ax=axes[int(idx / 4), idx % 4])
    axes[int(idx / 4), idx % 4].legend()
    axes[int(idx / 4), idx % 4].set_xlabel('churn')
    axes[int(idx / 4), idx % 4].set_ylabel(feat)
fig.savefig('boxplot.png')

<h3 style="text-align: justify">Найбільшу відмінність можна побачити для ознак *Total day minutes, Customer service calls* і *Number vmail messages*. Згодом ми навчимося визначати важливість ознак в задачі класифікації за допомогою випадкового лісу (або градієнтного бустінгу), і виявиться, що перші дві − дійсно дуже важливі ознаки для прогнозування відтоку.</h3>

<h3 style="text-align: justify">Подивимося окремо на картинки з розподілом тривалості розмов удень серед постійних та тих, які пішли. Зліва - знайомі нам боксплоти, праворуч − згладжені гістограми розподілу числових ознак в двох групах (швидше просто красива картинка, все і так зрозуміло по боксплоту).</h3>

<h3 style="text-align: justify">Цікаве спостереження: в середньому клієнти, які пішли більше користувалися зв'язком. Можливо, вони незадоволені тарифами, і одним із заходів боротьби з відтоком буде зниження тарифних ставок (вартості мобільного зв'язку). Але це вже компанії треба буде проводити додатковий економічний аналіз, чи дійсно такі заходи будуть виправдані.</h3>

In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(16,6))

sns.boxplot(x='churn', y='total day minutes', data=df, ax=axes[0]);
sns.violinplot(x='churn', y='total day minutes', data=df, ax=axes[1]);

<h3 style="text-align: justify">Тепер зобразимо розподіл числа звернень в сервісний центр. Тут унікальних значень ознаки небагато (ознаку можна вважати як кількісну цілочисельну, так і порядкову), і наочніше зобразимо розподіл за допомогою countplot. Спостереження: частка відтоку сильно зростає починаючи з 4 дзвінка в сервісний центр.</h3>

In [ ]:
sns.countplot(x='customer service calls', hue='churn', data=df);

<h3 style="text-align: justify">Тепер подивимося на зв'язок бінарних ознак *International plan і Voice mail plan* з відтоком. Спостереження: коли роумінг підключений, частка відтоку набагато вище, тобто наявність міжнародного роумінгу − сильна ознака. Про голосову пошту такого скзати не можна.</h3>

In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(16,6))

sns.countplot(x='international plan', hue='churn', data=df, ax=axes[0]);
sns.countplot(x='voice mail plan', hue='churn', data=df, ax=axes[1]);

<h3 style="text-align: justify">Нарешті, подивимося, як з відтоком пов'язана категоріальна ознака *state*. З нею вже не так приємно працювати, оскільки число унікальних штатів досить велике - 51. Можна для початку побудувати зведену табличку або порахувати відсоток відтоку для кожного штату. Але даних по кожному штату окремо замало (клієнтів, які пішли всього від 3 до 17 в кожному штаті), тому, можливо, ознаку *state* згодом не варто додавати в моделі класифікації через ризик *перенавчання* (але ми це будемо перевіряти на *крос-валідації*, stay tuned!).</h3>

<h3>Частки відтоку для кожного штату:</h3>

In [ ]:
df.groupby(['state'])['churn'].agg([np.mean]).sort_values(by='mean', ascending=False).T

<h3 style="text-align: justify">Видно, що в Нью-Джерсі і Каліфорнії частка відтоку вище 25%, а на Гаваях і в Алясці менше 5%. Але ці висновки побудовані на основі скромної статистики і можливо, це просто особливості наявних даних (тут можна перевірити гіпотези про кореляції Метьюса і Крамера, але це вже за рамками цієї лабораторної роботи).</h3>

<h3 style="text-align: justify">N-мірний простір з t-SNE</h3>

<h3 style="text-align: justify">Побудуємо t-SNE уявлення все тих же даних по відтоку. Назва методу складна − t-distributed Stohastic Neighbor Embedding, математика теж крута (і заглиблюватись в неї ми не будемо, але для бажаючих − [ось](http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf) оригінальна стаття Д. Хінтона і його аспіранта в JMLR), але основна ідея проста: знайдемо таке відображення з багатовимірного простору ознак на площину (або в 3D, але майже завжди вибирають 2D), щоб точки, які були далеко один від одного, на площині теж виявилися віддаленими, а близькі точки − також відобразилися на близькі. Тобто neighbor embedding − це свого роду пошук нового представлення даних, при якому зберігається сусідство.</h3>

<h3 style="text-align: justify">Трохи деталей: викинемо штати, номер телефону і ознаку відтоку, бінарні Yes/No-ознаки переведемо в числа (pd.factorize). Також потрібно масштабувати (нормалізувати) вибірку − від кожної ознаки відняти її середнє і поділити на стандартне відхилення, це робить StandardScaler.</h3>

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

In [ ]:
df.head()

In [ ]:
# Перетворимо ознаки international plan,voice mail plan в числові. 
# Відкинемо колонки: churn, state, phone number
X = df.drop(['churn', 'state','phone number'], axis=1)
X['international plan'] = pd.factorize(X['international plan'])[0]
X['voice mail plan'] = pd.factorize(X['voice mail plan'])[0]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

<h3 style="color: red;text-align: justify"> На слабких комп'ютерах операція може виконуватись доволі довго (наприклад у нас - 30 секунд). Слід трошки почекати!</h3>

In [ ]:
%%time
tsne = TSNE(random_state=17)
tsne_representation = tsne.fit_transform(X_scaled)

<h3 style="color: red;text-align: justify"> На слабких комп'ютерах графік може викликати зависання, тому за замовчуванням він закоментований.</h3>

In [ ]:
rcParams['figure.figsize'] = 10,7
#plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1]);

<h3 style="text-align: justify">Розфарбуємо отримане t-SNE уявлення даних по відтоку (сині - лояльні, помаранчеві – клієнти, які пішли).</h3>

<h3 style="color: red;text-align: justify"> На слабких комп'ютерах графік може викликати зависання, тому за замовчуванням він закоментований.</h3>

In [ ]:
#plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1], 
       # c=df['churn'].map({False: 'blue', True: 'orange'}));

<h3 style="text-align: justify">Бачимо, що клієнти, які пішли переважно перебувають в деяких областях простору ознак.</h3>

<h3 style="text-align: justify">Щоб краще зрозуміти картинку, можна також розфарбувати її за іншими бінарними ознаками − за роумінгом та голосовою поштою. Сині ділянки відповідають об'єктам, які мають ці бінарні ознаки.</h3>

<h3 style="color: red;text-align: justify"> На слабких комп'ютерах графік може викликати зависання, тому за замовчуванням він закоментований.</h3>

In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(16,6))

axes[0].scatter(tsne_representation[:, 0], tsne_representation[:, 1], 
            c=df['international plan'].map({'yes': 'blue', 'no': 'orange'}));
axes[1].scatter(tsne_representation[:, 0], tsne_representation[:, 1], 
            c=df['voice mail plan'].map({'yes': 'blue', 'no': 'orange'}));
axes[0].set_title('international plan');
axes[1].set_title('voice mail plan');

<h3 style="text-align: justify">Тепер зрозуміло, що, наприклад, багато клієнтів, які пішли перебувають в правому кластері людей з підключеним роумінгом, але без голосової пошти.</h3>

<h3>Зазначимо мінуси t-SNE:</h3>
<span style="font-size: 16px;text-align: justify">
* велика обчислювальна складність. [Ось](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) ця реалізація sklearn швидше за все не допоможе у Вашій реальній задачі, на великих вибірках варто подивитися в бік [Multicore-TSNE](https://github.com/DmitryUlyanov/Multicore-TSNE);
* картинка може сильно змінитися при зміні random seed, це ускладнює інтерпретацію. [Ось](C:\Users\Ivan\Documents\Institut\4 kurs\1 семестр\Моделі і системи\РГРМ\SAve\distill.pub\2016\misread-tsne\) хороший туторіал по t-SNE. Але в цілому по таким картинкам не варто робити далекосяжних висновків. Іноді щось кидається в очі і підтверджується при вивченні, але це не часто відбувається.
</span>

<h3 style="text-align: justify">За допомогою t-SNE можна дійсно отримати гарне уявлення про дані.</h3>